# Question Similarity with Sentence Transformer
The dataset used for training and testing is composed of Quora question sourced from Kaggle.

(https://www.kaggle.com/competitions/quora-question-pairs/)

In [2]:
!pip install sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which 

In [ ]:
!pip install num2words
!pip install spacy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 4.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=541e9adc0ebd23854b66d2c260dbcd6a7fdfe53b553a9d3567ba3cbd9ecae898
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Preparation

In [4]:
import pandas as pd


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/question similarity/train.csv')
df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df = df[['question1', 'question2', 'is_duplicate']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   question1     404289 non-null  object
 1   question2     404288 non-null  object
 2   is_duplicate  404290 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 9.3+ MB


In [ ]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404287 entries, 0 to 404289
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   question1     404287 non-null  object
 1   question2     404287 non-null  object
 2   is_duplicate  404287 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.3+ MB


In [ ]:
df['is_duplicate'].value_counts()

,count
is_duplicate,
0,255024
1,149263


split the dataset for training and testing

In [ ]:
# Sample 100000 rows for each class
df_0 = df[df['is_duplicate'] == 0].sample(n=100000, random_state=42)
df_1 = df[df['is_duplicate'] == 1].sample(n=100000, random_state=42)

# Concatenate the sampled data into the training DataFrame
train_df = pd.concat([df_0, df_1])


# The remaining rows will be used for the test DataFrame
test_df = df.drop(train_df.index)

train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
# Check the shapes of the resulting DataFrames
print(f"Train DataFrame shape: {train_df.shape}")
print(f"Test DataFrame shape: {test_df.shape}")

Train DataFrame shape: (200000, 3)
Test DataFrame shape: (204287, 3)


function to clean the text

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from num2words import num2words
from nltk.tag import pos_tag
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')


def preprocessing_data(text):
  text = text.lower()

  pattern = r'\b\d+\b'

  def replace_with_words(match):
      number = int(match.group())
      return num2words(number)

  text = re.sub(pattern, replace_with_words, text)

  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word not in stop_words]

  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  preprocessed_text = ' '.join(tokens)



  return preprocessed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train_df['question1'] = train_df['question1'].apply(preprocessing_data)
train_df['question2'] = train_df['question2'].apply(preprocessing_data)

test_df['question1'] = test_df['question1'].apply(preprocessing_data)
test_df['question2'] = test_df['question2'].apply(preprocessing_data)

In [ ]:
train_df.rename(columns={'question1': 'questionone'}, inplace=True)
train_df.rename(columns={'question2': 'questiontwo'}, inplace=True)
train_df.rename(columns={'is_duplicate': 'label'}, inplace=True)

test_df.rename(columns={'question1': 'questionone'}, inplace=True)
test_df.rename(columns={'question2': 'questiontwo'}, inplace=True)
test_df.rename(columns={'is_duplicate': 'label'}, inplace=True)

train_df.head(10)

,questionone,questiontwo,label
0,embarrassing thing happened high school,embarrassing thing happened high school,1
1,north korean defector react outside world,french colony,0
2,become expert coder,stratergy great coder,1
3,make person powerful speaker,make person powerful,0
4,would average person look like race merged one,whats like mistaken different ethnicity actually,0
5,good gift foreign visitor bring theyre invited...,good gift foreign visitor bring theyre invited...,0
6,importance battle somme,significance battle somme,1
7,conspiracy theory popular around world,people see conspiracy theory behind everything,1
8,control anger,one control hisher anger,1
9,mix group skilled worker construction project,construction worker use geometry work,0


In [ ]:
train_df.to_csv('/content/drive/MyDrive/dataset/question similarity/clean_train.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/dataset/question similarity/clean_test.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/question similarity/clean_train.csv')

In [ ]:
df.head(10)

,questionone,questiontwo,label
0,embarrassing thing happened high school,embarrassing thing happened high school,1
1,north korean defector react outside world,french colony,0
2,become expert coder,stratergy great coder,1
3,make person powerful speaker,make person powerful,0
4,would average person look like race merged one,whats like mistaken different ethnicity actually,0
5,good gift foreign visitor bring theyre invited...,good gift foreign visitor bring theyre invited...,0
6,importance battle somme,significance battle somme,1
7,conspiracy theory popular around world,people see conspiracy theory behind everything,1
8,control anger,one control hisher anger,1
9,mix group skilled worker construction project,construction worker use geometry work,0


In [ ]:
# Check for missing values
print(df.isnull().sum())
df = df.dropna()
print(df.isnull().sum())

questionone    32
questiontwo    30
label           0
dtype: int64
questionone    0
questiontwo    0
label          0
dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199946 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   questionone  199946 non-null  object
 1   questiontwo  199946 non-null  object
 2   label        199946 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 6.1+ MB


take take a small portion of the data because colab gpu is limited

In [ ]:
# Sample 100000 rows for each class
df_0 = df[df['label'] == 0].sample(n=25000, random_state=42)
df_1 = df[df['label'] == 1].sample(n=25000, random_state=42)

# Concatenate the sampled data into the training DataFrame
train_df = pd.concat([df_0, df_1])

train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
# Check the shapes of the resulting DataFrames
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   questionone  50000 non-null  object
 1   questiontwo  50000 non-null  object
 2   label        50000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


## Model Training

split data for training and validation

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import CoSENTLoss
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

model = SentenceTransformer(
    "msmarco-distilbert-base-tas-b",
)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(test_df)

train_columns = ['questionone', 'questiontwo', 'label']
test_columns = ['questionone', 'questiontwo', 'label']

train_dataset = train_dataset.select_columns(train_columns)
eval_dataset = eval_dataset.select_columns(test_columns)

loss = CoSENTLoss(model)

Specify training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/mpnet-base-custom-dataset",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="mpnet-base-custom-dataset",  # Will be used in W&B if `wandb` is installed
)

# Create an evaluator & evaluate the base model
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["questionone"],
    sentences2=eval_dataset["questiontwo"],
    scores=eval_dataset["label"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

dev_evaluator(model)

# Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

model.save_pretrained("/content/drive/MyDrive/dataset/question similarity/model1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine,Sts-dev Pearson Manhattan,Sts-dev Spearman Manhattan,Sts-dev Pearson Euclidean,Sts-dev Spearman Euclidean,Sts-dev Pearson Dot,Sts-dev Spearman Dot,Sts-dev Pearson Max,Sts-dev Spearman Max
100,3.787100,3.891796,0.506557,0.506922,0.487007,0.498594,0.487322,0.498941,0.522075,0.519297,0.522075,0.519297
200,3.869500,3.791490,0.514719,0.516980,0.498402,0.510681,0.498703,0.511016,0.525972,0.523173,0.525972,0.523173
300,3.710700,3.658529,0.529458,0.535821,0.519319,0.532518,0.519603,0.532845,0.533280,0.531453,0.533280,0.535821
400,3.605900,3.570876,0.542604,0.553933,0.538602,0.552412,0.538868,0.552702,0.539855,0.540264,0.542604,0.553933
500,3.554900,3.509556,0.553264,0.568439,0.553514,0.567928,0.553778,0.568205,0.544530,0.547236,0.553778,0.568439
600,3.448500,3.469326,0.564344,0.580948,0.568766,0.581382,0.568984,0.581582,0.548014,0.552705,0.568984,0.581582
700,3.488700,3.429760,0.572519,0.592014,0.581407,0.592720,0.581589,0.592827,0.549621,0.555390,0.581589,0.592827
800,3.487300,3.392380,0.580040,0.601926,0.590101,0.602871,0.590295,0.602991,0.551137,0.556782,0.590295,0.602991
900,3.383500,3.358495,0.587569,0.611885,0.600117,0.612999,0.600306,0.613112,0.552777,0.557770,0.600306,0.613112
1000,3.468800,3.331864,0.595760,0.618441,0.606625,0.619702,0.606818,0.619821,0.550422,0.554975,0.606818,0.619821


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [6]:
df_test = pd.read_csv('/content/drive/MyDrive/dataset/question similarity/clean_train.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   questionone  199968 non-null  object
 1   questiontwo  199970 non-null  object
 2   label        200000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.6+ MB


In [7]:
print(df_test.isnull().sum())
df_test = df_test.dropna()
print(df_test.isnull().sum())

questionone    32
questiontwo    30
label           0
dtype: int64
questionone    0
questiontwo    0
label          0
dtype: int64


## Model Testing

In [ ]:
from datasets import load_dataset, Dataset
from sklearn.metrics import f1_score
import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("/content/drive/MyDrive/dataset/question similarity/model1")

# Prepare the test data
question_one = df_test['questionone'].tolist()
question_two = df_test['questiontwo'].tolist()
labels = df_test['label'].tolist()

# Encode the questions using the model
embeddings_one = model.encode(question_one, convert_to_tensor=True)
embeddings_two = model.encode(question_two, convert_to_tensor=True)

# Calculate cosine similarities
cosine_scores = torch.nn.functional.cosine_similarity(embeddings_one, embeddings_two)

In [13]:
from scipy.stats import pearsonr, spearmanr

pearson_corr, _ = pearsonr(cosine_scores.cpu().numpy(), labels)
spearman_corr, _ = spearmanr(cosine_scores.cpu().numpy(), labels)

print(f"Pearson Correlation: {pearson_corr:.4f}")
print(f"Spearman Correlation: {spearman_corr:.4f}")

Pearson Correlation: 0.6809
Spearman Correlation: 0.7165
